In [580]:
using CSV, DataFrames

In [581]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "AI_factor_full_1997-2020.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2020.txt")

"/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full_1997-2020.txt"

In [582]:
AIS = CSV.read(inputAIS, DataFrame; header=true)
AIS[8157, [1, 2, 3, 20, 21, 22, 23, 24, 25]]

,Journal,1997,1998,2015,2016,2017,2018,2019,2020
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,0.789,0.789,0.772,0.868,0.655,0.595,0.458,0.409


In [583]:
ISI = CSV.read(inputISI, DataFrame; header=true)
ISI[8157, [1, 2, 3, 20, 21, 22, 23, 24, 25]]

,Journal,1997,1998,2015,2016,2017,2018,2019,2020
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,1.692,1.692,1.978,1.644,1.568,1.672,1.54,1.549
